In [1]:
import json
from bs4 import BeautifulSoup
from html.parser import HTMLParser
from collections import defaultdict

Сохранение словаря: номер запроса: номера документов 

In [ ]:
import json
queryDocs = {}
for numb in numbQueries:
    queryDocs[int(numb)] = list(map(int, list(groupSample.get_group(numb)["DocumentId"])))    
json.dump(queryDocs, open(queriesDocsPath, 'w'))

In [ ]:
class PageText:
    def __init__(self, title='', description='', keywords='', textDir={}, attrDir={}, number=-1, url=''):
        self._title = title
        self._description = description
        self._keywords = keywords
        self._textDir = textDir
        self._attrDir = attrDir
        self._number = number
        self._url = url
        
    def setNumber(self, number):
        self._number = number
        
    def setUrl(self, url):
        self._url = url
        
    def toDict(self):
        return {"number":self._number,
                "url":self._url,
                "title":self._title, 
                "description":self._description, 
                "keywords": self._keywords,
                "text": self._textDir,
                "attr": self._attrDir}

In [3]:
class TextHTMLParser(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self._textDir = defaultdict(list)
        self._interestAttrs = ['title', 'alt']
        self._attrDir = defaultdict(lambda: dict.fromkeys(self._interestAttrs, []))
        
        self._title = ''
        self._description = ''
        self._keywords = ''        
        
        self.__curTag = None
               
    def handle_starttag(self, tag, attrs):
        attrs_dict = dict(attrs)
        if tag == 'meta':            
            name = attrs_dict.get('name', '')
            if name == 'description':
                self._description = attrs_dict.get('content', '').strip()
            if name == 'keywords':
                self._keywords = attrs_dict.get('content', '').strip()   
                
        for attr in self._interestAttrs:
            value = attrs_dict.get(attr, '').strip()
            if value:
                self._attrDir[tag][attr].append(value) 
        self.__curTag = tag
        
    def handle_data(self, data):
        if data.strip():
            data = data.strip()
            if self.__curTag == 'title':
                self._title = self._title + data
            elif self.__curTag != 'script':
                self._textDir[self.__curTag].append(data)
    def pageText(self):
        return PageText(self._title, self._description, self._keywords, self._textDir, self._attrDir)

    def handle_endtag(self, tag):
        pass
    def clear(self):
        self._textDir = defaultdict(list)
        self._attrDir = defaultdict(lambda: dict.fromkeys(self._interestAttrs, []))        
        self._title = ''
        self._description = ''
        self._keywords = ''                
        self.__curTag = None

In [4]:
dataPath = '../data'
docUrlsName = '../data/urls.docs.txt'
textdataPath = '../data/textdata'
templateJson = '{:d}.json'

In [5]:
docDict = {}
with open(docUrlsName) as inputFile:
    for line in inputFile:
        number, url, path = line.strip().split('\t')
        docDict[int(number)] = (url, path)

# Парсим документы(создаем json файлы с текстом и значения некоторых атрибутов, таких как title и alt

In [57]:
def tryParse(parser, fileInput):
    try:
        parser.clear()
        parser.feed(fileInput.read())
        return True
    except Exception:
        return False

In [58]:
encods = ['utf-8', 'windows-1251']
for number, (url, path) in docDict.items():
    relativePath = '{:s}/{:s}'.format(dataPath, path)
    outputPath = '{:s}/{:s}'.format(textdataPath, templateJson.format(number))    
    for encod in encods:
        parser = TextHTMLParser()
        fileInput = open(relativePath, encoding=encod)
        ans = tryParse(parser, fileInput)
        if ans:
            page = parser.pageText()
            page.setNumber(number)
            page.setUrl(url)
            json.dump(page.toDict(), open(outputPath, 'w'))
            fileInput.close()
            break
        fileInput.close()       

Документы, которые не распарсились

In [61]:
import os
import re
pat = re.compile(r'\d+')
procNumbs = set( map(lambda x: int(pat.search(x).group(0)), os.listdir(textdataPath)))
allNumbs = set(docDict.keys())
errorNumbs = allNumbs.difference(procNumbs)

In [65]:
print(len(errorNumbs))

1587
